In [ ]:
%%HTML
<style>
.container { width:100% !important }
</style>

# The $3 \times 3$ Sliding Puzzle

<img src="8-puzzle.png">

The picture above shows an instance of the $3 \times 3$ 
<a href="https://en.wikipedia.org/wiki/Sliding_puzzle">sliding puzzle</a>:
There is a board of size $3 \times 3$ with 8 tiles on it. These tiles are numbered with digits from the set $\{1,\cdots, 8\}$.  As the the $3 \times 3$ board has an area of $9$ but there are only $8$ tiles, there is an empty square on the board.  Tiles adjacent to the empty square can be moved into the square, thereby emptying the space that was previously occupied by theses tiles.  The goal of the $3 \times 3$ puzzle is to transform the state shown on the left of the picture above into the state shown on the right.

In order to get an idea of the sliding puzzle, you can play it online at <a href="http://mypuzzle.org/sliding">http://mypuzzle.org/sliding</a>.

## Utilities for Displaying the Solution

In [ ]:
Colors = ['white', 'lightblue', 'pink', 'magenta', 
          'orange', 'red', 'yellow', 'lightgreen', 'salmon'
         ]

In [ ]:
def get_style(n):
    return 'background-color: ' + Colors[n] + ';">'

In [ ]:
CSS_Table = { 'border'      : '2px solid darkblue',
              'border-style': 'double',
              'border-width': '4px'
            }

In [ ]:
CSS_TD = { 'border'      : '2px solid black', 
           'border-style': 'groove',
           'border-width': '8px',
           'padding'     : '15px',
           'font-size'   : '150%', 
         }

In [ ]:
def css_style(Dictionary):
    result = ''
    for k, v in Dictionary.items():
        result += k + ':' + v + ';'
    return result

In [ ]:
def state_to_html(State):
    result = '<table style="' + css_style(CSS_Table) + '">\n'
    for row in State:
        result += '<tr>'
        for number in row:
            result += '<td style="' + css_style(CSS_TD)
            if number > 0:
                result += get_style(number) + str(number)
            else:
                result += get_style(number)
            result += '</td>'
        result += '</tr>\n'
    result += '</table>'
    return result

## Computing a Path 

The functions given below are identical to the functions that we have used to solve the shunting-yard problem from the previous exercise.

In [ ]:
def pathProduct(P, R):
    "Attach all possible pairs from R at the end of the paths from P."
    return { L + (b,) for L in P for (a, b) in R 
                      if L[-1] == a and not b in L 
           }

In [ ]:
def arb(S):
    "Return an arbitrary element from the set S but do not remove it."
    for x in S:
        return x

The sliding puzzle has $9! = 362\,880$ different states and the transition relation $R$ has a size of $846\,720$ pairs.  Keeping the relation $R$ in memory is going to be slow.  Instead, we will implement a function $\texttt{nextStates}(S)$ that takes a state $S$ as input and computes the set of all states that can be reached from $S$.  For any given state $S$ the set $\texttt{nextStates}(S)$ has at most four elements.  Accordingly I had to change the definition of the function <tt>findPath</tt>.  This function is now called as $\texttt{findpath}(\texttt{start},\texttt{goal}, \texttt{nextStates})$ where <tt>start</tt> and <tt>goal</tt> are states and $\texttt{nextStates}$ is a function.   It implements the 
<em style="color:blue">breadth first search</em> algorithm.

In [ ]:
def findPath(start, goal, nextStates):
    count    = 0  # the length of the paths explored
    Paths    = { (start,) }
    States   = { start }   # states known to be reachable
    Explored = {}          # states that have already been explored
    while States != Explored:
        Explored = States.copy()     # copy to avoid aliasing bug
        Paths    = { P + (s,) for P in Paths
                              for s in nextStates(P[-1])
                              if  s not in States # keep only paths to new states
                   }
        States  |= { P[-1] for P in Paths } # set of all states visited so far
        count   += 1
        print("Number of states with distance %3s: %6s" % (count, len(States)))
        if goal in States: 
            return arb({ P for P in Paths if P[-1] == goal })

## Problem Specific Code

We will represent states as tuples of tuples.  For example, the start state that is shown in the picture at the beginnning of this notebook is represented as follows:

In [ ]:
start = ((8, 0, 6),
         (5, 4, 7),
         (2, 3, 1)
        )

Note that the empty tile is represented by the digit $0$.  

**Exercise 1**: Define the goal state below.

In [ ]:
goal = "your code here"

**Exercise 2:** 
The function $\texttt{findZero}(S)$ takes a state $S$ and returns a pair $(r, c)$ that specifies the row and the column of the blank in the state $S$.  For example, we should have:
$$ \texttt{findZero}(\texttt{start}) = (0, 1) \quad\mbox{and}\quad 
   \texttt{findZero}(\texttt{goal}) = (0, 0) 
$$

In [ ]:
def findZero(S):
    M = {0, 1, 2}
    P = {(r, c) for r in M
                for c in M 
                if S[r][c] == 0
        }
    return arb(P)

In [ ]:
findZero(start)

In [ ]:
findZero(goal)

We have to represent states as tuples of tuples in order to be able to insert them into sets.  However, as tuples are immutable, we need to be able to convert them to lists in order to change them.  The function $\texttt{listOfLists}(S)$ takes a state $S$ and transforms it into a list of lists.

In [ ]:
def listOfLists(S):
    "Transform a tuple of tuples into a list of lists."
    return [ [x for x in row] for row in S ]

In [ ]:
listOfLists(start)

As lists can not be inserted into sets, we also need a function that takes a list of list and transforms it into a tuple of tuple.

In [ ]:
def tupleOfTuples(S):
    "Transform a list of lists into a tuple of tuples."
    return tuple(tuple(x for x in row) for row in S)

In [ ]:
tupleOfTuples([[8, 0, 6], [5, 4, 7], [2, 3, 1]])

**Exercise 3**: Implement a function $\texttt{moveUp}(S, r, c)$ that computes the state that results from moving the tile below the blank space **up** in state $S$.  The variables $r$ and $c$ specify the location of the *row* and *column* of the blank tile.  Therefore we have $S[r][c] = 0$.

In your implementation you may assume that there is indeed a tile below the blank space, i.e. we have $r < 2$.  

In [ ]:
def moveUp(S, r, c):
    "Move the tile below the blank up."
    T = listOfLists(S)
    "your code here"
    return tupleOfTuples(T)

In [ ]:
moveUp(start, 0, 1)

**Exercise 4**: Implement a function $\texttt{moveDown}(S, r, c)$ that computes the state that results from moving the tile below the blank space **down** in state $S$.  The variables $r$ and $c$ specify the location of the *row* and *column* of the blank tile.  Therefore we have $S[r][c] = 0$.

In your implementation you may assume that there is indeed a tile above the blank space, i.e. we have $r > 0$.  

In [ ]:
def moveDown(S, r, c):
    "Move the tile above the blank down."
    "your code here"

**Exercise 5:**
Similarly to the previous exercise, implement functions $\texttt{moveRight}(S, r, c)$ and $\texttt{moveLeft}(S, r, c)$.

In [ ]:
def moveRight(S, r, c):
    "Move the tile left of the blank to the right."
    "your code here"

In [ ]:
def moveLeft(S, r, c):
    " Move the tile right of the blank to the left."
    "your code here"

**Exercise 6:**. Implement a function $\texttt{nextStates}(S)$ that takes a state $S$ representet as a tuple of tuple and that computes the set of states that are reachable from $S$ in one step.  Remember to use the previously defined functions $\texttt{findZero}$, $\texttt{moveUp}$, $\cdots$, $\texttt{moveLeft}$.  However, when you do use the function $\texttt{moveUp}$, then you should also check that it is possible to move a tile up.

In [ ]:
def nextStates(S):
    "Compute the set of states reachable in one step from the state S."
    "your code here"

The following computation takes about 10 seconds on my desktop computer, which has an 3,4 GHz Quad-Core Intel Core i5 (7500) Prozessor.  The final number of states touched is $181\,440$.  

In [ ]:
%%time
Path = findPath(start, goal, nextStates)

The tuple Path that is a solution to the sliding problem has a length of **32**.  If your path is shorter, then you have to inspect it carefully to identify the problem.  In order to do this, use the function <tt>printPath</tt> that is implemented at the bottom of this notebook.

In [ ]:
len(Path)

Print the solution via `HTML` tables.

In [ ]:
for State in Path:
    display(HTML(state_to_html(State)))